In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
lb = pd.read_csv('/content/original/dayLifeBeam.csv')

lb = lb.round(2)

lb.to_csv('dayLifeBeamNew.csv', index=False)


In [ ]:
lb['CNO'] = lb['CNO'].astype(float)

In [ ]:
lb['hKena'] = np.where(
    (lb['sisaCalories'] > 0) & (lb['BMI'] >= 25),
    0,
    np.where(
        (lb['sisaCalories'] > 0) & (lb['BMI'] < 25),
        lb['CNO'] / lb['sisaCalories'],
        np.where(
            (lb['sisaCalories'] <= 0) & (lb['BMI'] <= 18.49),
            0,
            lb['CLU'] / lb['sisaCalories']
        )
    )
)

In [ ]:
lb.to_csv('CleaneddayDATA.csv', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('CleaneddayDATA.csv')

df = df.round(2)

df.to_csv('roundedCleaneddayDATA.csv', index=False)

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error
import pickle

data = pd.read_csv('roundedCleaneddayDATA.csv')

X = data.drop(columns=['hKena'])
y = data['hKena']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

scaler_filename = 'hKenaScalerBeta.pkl'
pickle.dump(scaler, open(scaler_filename, 'wb'))

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

history = model.fit(X_train_scaled, y_train, epochs=1000, batch_size=32, validation_data=(X_test_scaled, y_test))

model.save('hKenaModelBeta.h5')

predictions = model.predict(X_test_scaled)
r2 = r2_score(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)

print("R2 Score:", r2)
print("MAE:", mae)

Epoch 1/1000
3/3 [==============================] - 4s 176ms/step - loss: 135749.0625 - mae: 210.1292 - val_loss: 31210.7227 - val_mae: 124.6719
Epoch 2/1000
3/3 [==============================] - 0s 24ms/step - loss: 135735.7188 - mae: 210.1258 - val_loss: 31208.5410 - val_mae: 124.6654
Epoch 3/1000
3/3 [==============================] - 0s 36ms/step - loss: 135752.5938 - mae: 210.1400 - val_loss: 31206.3789 - val_mae: 124.6597
Epoch 4/1000
3/3 [==============================] - 0s 46ms/step - loss: 135712.9062 - mae: 210.1246 - val_loss: 31204.3125 - val_mae: 124.6545
Epoch 5/1000
3/3 [==============================] - 0s 53ms/step - loss: 135787.8438 - mae: 210.1690 - val_loss: 31202.3945 - val_mae: 124.6501
Epoch 6/1000
3/3 [==============================] - 0s 59ms/step - loss: 135689.0938 - mae: 210.1067 - val_loss: 31200.5566 - val_mae: 124.6460
Epoch 7/1000
3/3 [==============================] - 0s 40ms/step - loss: 135764.7969 - mae: 210.1338 - val_loss: 31198.6660 - val_mae: 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 188ms/step
R2 Score: 0.1732003508746297
MAE: 88.46612419128418


In [ ]:
model = tf.keras.models.load_model('/content/hKenaModelBeta.h5')

scaler_filename = '/content/hKenaScalerBeta.pkl'
scaler = pickle.load(open(scaler_filename, 'rb'))

new_data = {
    'BMI': 18,
    'CNO': 	194425,
    'CLU': 	0,
    'sisaCalories': -995.91,
}

new_df = pd.DataFrame([new_data])

new_data_scaled = scaler.transform(new_df)

prediction = model.predict(new_data_scaled).flatten()[0]

print('Predicted hKena:', prediction)


1/1 [==============================] - 0s 128ms/step
Predicted hKena: 80.38464
